<a href="https://colab.research.google.com/github/AtinChing/RoBERTa-Finetuning-Strategies-Analysis/blob/main/LoRA_fine_tuning_of_RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install "transformers>=4.44" datasets peft accelerate evaluate scikit-learn bitsandbytes pynvml
print("✅ Dependencies installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.1 MB/s eta 0:00:00
✅ Dependencies installed


In [ ]:
import os, time, json
import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import evaluate

try:
    import pynvml
    pynvml.nvmlInit()
    NVML_AVAILABLE = True
except:
    NVML_AVAILABLE = False

print("✅ Imports loaded")


✅ Imports loaded


In [ ]:
# ================================
# ✅ CELL 4 — Load IMDB Movie Reviews Dataset
# ================================

MODEL_NAME = "roberta-base"
MAX_LEN = 256

from datasets import load_dataset
from transformers import AutoTokenizer
import collections

# 1. Load dataset from Hugging Face Hub ("imdb" = 50k movie reviews)
dataset = load_dataset("imdb")

# 2. Load tokenizer for RoBERTa
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# 3. Tokenization function (convert text → input_ids + attention_mask)
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=MAX_LEN)

# Apply tokenization and remove original text column
dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

# Rename 'label' → 'labels' because Trainer expects `labels`
dataset = dataset.rename_column("label", "labels")

# Format dataset as PyTorch tensors
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


# ================================
# ✅ VERIFICATION CHECKS
# ================================

print("✅ Dataset structure:")
print(dataset)                # shows train/test split + feature columns
print("\n🔍 Example row (train[0]):")
print(dataset["train"][0])   # prints one sample with labels/token IDs

# Check label distribution (sentiment balance)
label_counts = collections.Counter(dataset["train"]["labels"])
print("\n📊 Label distribution (0 = negative, 1 = positive):")
print(label_counts)

print("\n🎬 Data verified — this IS the IMDB movie review dataset.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

✅ Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

🔍 Example row (train[0]):
{'labels': tensor(0), 'input_ids': tensor([    0,   100, 16425,    38,  3326,   230, 42338, 18024,    12,   975,
        25322,  4581,    31,   127,   569,  1400,   142,     9,    70,     5,
         6170,    14,  7501,    24,    77,    24,    21,    78,   703,    11,
        13025,     4,    38,    67,  1317,    14,    23,    78,    24,    21,
         5942,    30,   121,     4,   104,     4, 10102,   114,    24,   655,
         1381,     7,  2914,    42,   247,     6,  3891,   145,    10,  2378,
            9,  3541,  1687,    22, 10800, 34689,   113,    38,   269,    56,
         

In [ ]:
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2, torch_dtype=dtype
)

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query","key","value"]  # tell LoRA where to inject adapters
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("✅ RoBERTa modified with LoRA adapters")


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 442,368 || all params: 125,089,538 || trainable%: 0.3536
✅ RoBERTa modified with LoRA adapters


In [ ]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
roc = evaluate.load("roc_auc")

def softmax_np(x):
    x = x - np.max(x, axis=-1, keepdims=True)
    ex = np.exp(x)
    return ex / np.sum(ex, axis=-1, keepdims=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = softmax_np(logits)
    preds = np.argmax(probs, axis=-1)

    out = {}
    out.update(accuracy.compute(predictions=preds, references=labels))
    out.update(f1.compute(predictions=preds, references=labels, average="weighted"))
    try:
        out.update(roc.compute(prediction_scores=probs[:,1], references=labels))
    except:
        out["roc_auc"] = float("nan")
    return out

from transformers.trainer_callback import TrainerCallback

class GPUTracker(TrainerCallback):
    def __init__(self):
        self.util = []
    def on_step_end(self, *args, **kwargs):
        if NVML_AVAILABLE:
            h = pynvml.nvmlDeviceGetHandleByIndex(0)
            util = pynvml.nvmlDeviceGetUtilizationRates(h)
            self.util.append(util.gpu)

gpu_tracker = GPUTracker()

print("✅ Metrics + GPU monitoring ready")


✅ Metrics + GPU monitoring ready


In [ ]:
# ================================
# ✅ CELL 7 — Train LoRA RoBERTa (version-safe TrainingArguments)
# ================================
import time, inspect
from transformers import TrainingArguments, Trainer

OUT_DIR = "./lora_roberta_imdb"

# Figure out which kwarg name your transformers version expects
sig = inspect.signature(TrainingArguments.__init__)
arg_names = set(sig.parameters.keys())
EVAL_KEY = "evaluation_strategy" if "evaluation_strategy" in arg_names else ("eval_strategy" if "eval_strategy" in arg_names else None)
if EVAL_KEY is None:
    raise RuntimeError("Your transformers version doesn't accept evaluation_strategy or eval_strategy.")

# Build kwargs common to all versions
ta_kwargs = dict(
    output_dir=OUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_ratio=0.06,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=(dtype == torch.float16),
    bf16=(dtype == torch.bfloat16),
    logging_steps=50,
    report_to="none",
    seed=42,
)

# Inject the correct eval strategy key
ta_kwargs[EVAL_KEY] = "epoch"

training_args = TrainingArguments(**ta_kwargs)

# If you defined GPUTracker in the previous cell, reuse it; otherwise comment callbacks out.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
    callbacks=[gpu_tracker]  # requires GPUTracker from Cell 6
)

# Train + evaluate
t0 = time.time()
trainer.train()
train_time = time.time() - t0

eval_metrics = trainer.evaluate()
print("✅ Training complete")
print("⏱️ Train time (sec):", round(train_time, 2))
print("📈 Eval metrics:", eval_metrics)


✅ Training complete
⏱️ Train time (sec): 1682.45
📈 Eval metrics: {'eval_loss': 0.18924713134765625, 'eval_accuracy': 0.93012, 'eval_f1': 0.9301199945214076, 'eval_roc_auc': 0.9802334304, 'eval_runtime': 518.5029, 'eval_samples_per_second': 48.216, 'eval_steps_per_second': 1.508, 'epoch': 1.0}


In [ ]:
# ================================
# ✅ CELL 8 — Save Results + LoRA Adapter
# ================================
import os, json

# Collect metrics (already computed in Cell 7)
results = {
    "accuracy": eval_metrics["eval_accuracy"],
    "f1": eval_metrics["eval_f1"],
    "roc_auc": eval_metrics["eval_roc_auc"],
    "train_time_sec": round(train_time, 2),
    "eval_loss": eval_metrics["eval_loss"],
}

os.makedirs(OUT_DIR, exist_ok=True)

# Save LoRA adapter (only the trainable weights)
trainer.model.save_pretrained(os.path.join(OUT_DIR, "adapter"))

# Save JSON result file
with open(os.path.join(OUT_DIR, "results.json"), "w") as f:
    json.dump(results, f, indent=2)

# Save CSV result file (for graphing later)
with open(os.path.join(OUT_DIR, "results.csv"), "w") as f:
    f.write("accuracy,f1,roc_auc,train_time_sec,eval_loss\n")
    f.write(f"{results['accuracy']},{results['f1']},{results['roc_auc']},{results['train_time_sec']},{results['eval_loss']}")

print("✅ Saved all outputs!")
print(f"📁 Adapter directory: {OUT_DIR}/adapter/")
print(f"📄 JSON results:      {OUT_DIR}/results.json")
print(f"📄 CSV results:       {OUT_DIR}/results.csv")


✅ Saved all outputs!
📁 Adapter directory: ./lora_roberta_imdb/adapter/
📄 JSON results:      ./lora_roberta_imdb/results.json
📄 CSV results:       ./lora_roberta_imdb/results.csv
